<a href="https://colab.research.google.com/github/XiaoHuang0803/ColabNotebooks/blob/main/001_LLM_Appendix_A_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.__version__

'2.4.1+cu121'

In [2]:
torch.cuda.is_available()

True

In [3]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
loss

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)

In [4]:
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [5]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [6]:
torch.manual_seed(2024)
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [7]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params

2213

In [8]:
model.layers[0].weight

Parameter containing:
tensor([[ 0.0090,  0.0937,  0.1335,  ...,  0.1330, -0.0752,  0.0450],
        [ 0.0489, -0.0849, -0.0655,  ..., -0.0295, -0.1382,  0.0253],
        [-0.0789,  0.0079, -0.0172,  ..., -0.0404,  0.0713,  0.1166],
        ...,
        [-0.0576, -0.1120, -0.1377,  ..., -0.0457, -0.0412,  0.1293],
        [ 0.0668,  0.1049, -0.0159,  ...,  0.0209,  0.0283, -0.0486],
        [ 0.1056,  0.0160, -0.1210,  ...,  0.0966,  0.0493,  0.1363]],
       requires_grad=True)

In [9]:
X = torch.rand(1, 50)
out = model(X)
print(out)

tensor([[-0.1827,  0.0583, -0.0637]], grad_fn=<AddmmBackward0>)


In [11]:
# no training:
with torch.no_grad():
    out = model(X)
    out_prob = torch.softmax(out, dim=1)
    print(out)
    print(out_prob)

tensor([[-0.1827,  0.0583, -0.0637]])
tensor([[0.2942, 0.3744, 0.3314]])


In [12]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0, 1])

In [14]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [15]:
print(len(train_ds))

5


In [21]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [22]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}: ", x, y)

Batch 1:  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2:  tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


In [23]:
# A typical training loop
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

num_epochs = 3
for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train Loss: {loss: .2f}")

    model.eval()

Epoch: 001/003 | Batch 000/002 | Train Loss:  0.75
Epoch: 001/003 | Batch 001/002 | Train Loss:  0.65
Epoch: 002/003 | Batch 000/002 | Train Loss:  0.44
Epoch: 002/003 | Batch 001/002 | Train Loss:  0.13
Epoch: 003/003 | Batch 000/002 | Train Loss:  0.03
Epoch: 003/003 | Batch 001/002 | Train Loss:  0.00


In [24]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [26]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
predictions = torch.argmax(probas, dim=1)
print(probas)
print(predictions)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])
